In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
import torch

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish-medium")
#model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish-medium")
model =AutoModelForCausalLM.from_pretrained("LuisDavidFT777/gpt2-spanish-medium")

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [29]:
dfC = 'E:/chatbot CEPI/data/pregunta_respuesta.csv'
dfClean = pd.read_csv(dfC)


In [30]:
dfClean=dfClean[1000:1300].reset_index()

In [31]:
print('pregunta: ---- '+dfClean.loc[10,'pregunta'])
print('respuesta: ---- '+dfClean.loc[10,'respuesta'])

pregunta: ---- <nombre> diplomado en derecho procesal penal, versión i, modalidad virtual <monto> 760.0 <costo> 3000
respuesta: ---- <titulo> bachiller en humanidades <area> humanidades y sociales <universidad> mayor, real y pontificia de san francisco xavier


In [32]:
textTokenizar =  '<humano> Dame una recomendacion de este curso '+dfClean['pregunta']+' <asistente> '+dfClean['respuesta']

In [33]:
tokenized_dataset =[]
att_mask = []
for i in range(0,len(dfClean['pregunta'])):
    wordToken = tokenizer(textTokenizar[i],padding='max_length',truncation=True,return_tensors='pt', max_length=100)
    tokenized_dataset.append(wordToken['input_ids'][0])
    att_mask.append(wordToken['attention_mask'][0])

In [37]:
inp = {'input_ids':tokenized_dataset,'text':textTokenizar, 'attention_mask':att_mask}

In [38]:
from datasets import Dataset
data = Dataset.from_dict(inp)

In [39]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./finetuned_model',
    num_train_epochs=1,  # Número de épocas de entrenamiento
    per_device_train_batch_size=8,  # Tamaño del lote de entrenamiento por dispositivo
    logging_steps=1,
    logging_first_step=True,
    remove_unused_columns=True
)

# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    data_collator=data_collator
)

In [40]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=38, training_loss=0.3539005117981057, metrics={'train_runtime': 1243.3002, 'train_samples_per_second': 0.241, 'train_steps_per_second': 0.031, 'total_flos': 54416056320000.0, 'train_loss': 0.3539005117981057, 'epoch': 1.0})

In [41]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'text', 'attention_mask'],
    num_rows: 300
})

In [42]:
model.push_to_hub('LuisDavidFT777/gpt2-spanish-medium', token=True)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

E:\chatbot CEPI\venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--LuisDavidFT777--gpt2-spanish-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuisDavidFT777/gpt2-spanish-medium/commit/df82c4fd98e3bd14ac505c3941364741a7523d40', commit_message='Upload model', commit_description='', oid='df82c4fd98e3bd14ac505c3941364741a7523d40', pr_url=None, pr_revision=None, pr_num=None)

In [43]:
model.save_pretrained('gpt2_2')